In [19]:
model_file = "../results/"
queries_file = "../results/"
results_file = "../results/"

with open("../data/busca.cfg", "r") as config_file:
    for line in config_file.readlines():
        instruction, filename = line.split("=")
        filename = filename.strip()
        
        if instruction == "MODELO":
            model_file += filename
        elif instruction == "CONSULTAS":
            queries_file += filename
        elif instruction == "RESULTADOS":
            results_file += filename

In [20]:
import pandas as pd

model = pd.read_csv(model_file, sep=";")
model.set_index(["Token"], inplace=True)

In [21]:
import generate_inverted_list as gil

queries = pd.read_csv(queries_file, sep=";")
queries.set_index(["QueryNumber"], inplace=True)
for number, text in queries.itertuples():
    processed_text = gil.preproccess_text(text)
    queries.at[number, "QueryText"] = processed_text

In [24]:
# Add queries no modelo
import numpy as np

shape = (model.shape[0], 1)
for qnumber, qtext in queries.itertuples():
    zeros = pd.DataFrame(np.zeros(shape), index=model.index, columns=[f"Q{qnumber}"])
    model = pd.concat([model, zeros], axis=1)
    for word in qtext:
        if not word in model.index:
            zeros = pd.DataFrame(np.zeros((1, len(model.columns))), index=[word], columns=model.columns)
            model = pd.concat([model, zeros], axis=0)
            shape = (model.shape[0], 1)
        model.at[word, f"Q{qnumber}"] += 1

In [6]:
def get_vector(document, model):
    return model[str(document)].to_numpy()  

In [7]:
def get_vector_size(vector):
    return np.linalg.norm(vector)

In [8]:
def sim_cos(query, document, model):
    q = get_vector(query, model)
    d = get_vector(document, model)

    q_dot_d = np.dot(q, d)
    qxd = get_vector_size(q) * get_vector_size(d)
    
    return q_dot_d / qxd

In [27]:
ranking = pd.DataFrame()
shape = (model.shape[1] - 99, 1)
for query in queries.index:
    q = f"Q{query}"
    zeros = pd.DataFrame(np.zeros(shape), index=model.columns[:-99], columns=[q])
    ranking = pd.concat([ranking, zeros], axis=1)
    # Iterar pelas colunas de documento, retirando as de query
    for document in model.columns[:-99]:
        result = sim_cos(q, document, model)
        ranking.loc[document, q] = result

In [28]:
model

,1,6,19,24,30,47,52,53,54,62,...,Q90,Q91,Q92,Q94,Q95,Q96,Q97,Q98,Q99,Q100
SIGNIFIC,0.126373,0.084249,0.068931,0.084249,0.151648,0.126373,0.252747,0.252747,0.37912,0.126373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PSEUDOMONA,0.613622,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.511352,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AERUGINOSA,0.607672,0.810230,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.101279,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
INFECT,0.485316,0.323544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.727975,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RESPIRATORI,0.535870,0.238164,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MONOSPECIF,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CONSENT,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PATCHI,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
THEORIZ,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
def get_results(file, ranking):
    with open(file, 'w') as results:
        results.write("QueryNumber;DocInfos\n")
        for query in ranking.columns:
            query_number = query.replace('Q', '')
            sorted_ranking = ranking[query].sort_values(ascending=False)
            position_ranking = 1
            for doc_number, cos in sorted_ranking.items():
                if cos == 0:
                    break
                doc_infos = [position_ranking, doc_number, cos]
                position_ranking += 1
                results.write(f"{query_number};{doc_infos}\n")

In [14]:
get_results(results_file, ranking)